In [1]:
import joule
import requests
import json

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


In [2]:
node = joule.api.get_node()
root = await node.folder_get('/')

# flush the database
for child in root.children:
    await node.folder_delete(child,recursive=True)
root = await node.folder_get('/')
root = await node.folder_get('/')

print(f"{len(root.children)} top level folders")


0 top level folders


In [3]:
async def save_schema(filename):
    master = await node.master_add(master_type="user",identifier="jupyter")
    resp = requests.get("https://127.0.0.1:8088/folders.json", verify=False,
                 headers={"X-API-KEY": master.key})
    await node.master_delete(master_type="user",name="jupyter")
    
    with open(filename,"w") as f:
        f.write(json.dumps(resp.json(),indent=2))

### Test Database Schema: 

<pre>
root
├── folder_1
│   ├── stream_1_1: float32_3
│   └── stream_1_2: uint8_3
├── folder_2
│   └── stream_2_1: int16_2
│   └── transients (event stream)
│   └── loads (event stream)
├── folder_3
│   ├── folder_3_1
│   │   └── stream_3_1_1: int32_3
│   └── stream_3_1: uint16_3
└── folder_4
    └── folder_4_1
</pre>


In [4]:

stream_1_1 = joule.api.DataStream("stream_1_1",elements=[joule.api.Element(x) for x in "xyz"])
stream_1_1.elements[0].default_max=100
stream_1_1.elements[0].display_type="continuous"
stream_1_1.elements[1].default_min=-6
stream_1_1.elements[1].display_type="event"
stream_1_1.elements[2].units="watts"
stream_1_1.elements[2].display_type="discrete"

stream_1_2 = joule.api.DataStream("stream_1_2",datatype="uint8",elements=[joule.api.Element(f"{i}") for i in range(3)])
stream_2_1 = joule.api.DataStream("stream_2_1",datatype="int16",elements=[joule.api.Element(f"{i}") for i in range(2)])
stream_3_1 = joule.api.DataStream("stream_3_1",datatype="uint16",elements=[joule.api.Element(f"{i}") for i in range(3)])
stream_3_1_1 = joule.api.DataStream("stream_3_1_1",datatype="int32",elements=[joule.api.Element(f"{i}") for i in range(3)])
stream_4_1_1 = joule.api.DataStream("stream_4_1",datatype="uint16",elements=[joule.api.Element(f"{i}") for i in range(3)])
transient_events = joule.api.EventStream("transients")
load_events = joule.api.EventStream("loads")
stream_1_1 = await node.data_stream_create(stream_1_1,"/folder_1")
stream_1_2 = await node.data_stream_create(stream_1_2,"/folder_1")
stream_2_1 = await node.data_stream_create(stream_2_1,"/folder_2")
stream_3_1 = await node.data_stream_create(stream_3_1,"/folder_3")
stream_3_1_1 = await node.data_stream_create(stream_3_1_1,"/folder_3/folder_3_1")
transient_events =await node.event_stream_create(transient_events, "/folder_2")
load_events = await node.event_stream_create(load_events, "/folder_2")

stream_4_1_1 = await node.data_stream_create(stream_4_1_1,"/folder_4/folder_4_1")
await node.data_stream_delete(stream_4_1_1)

await save_schema("0_original_schema.json")

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


*Update element name, data stream name, and event stream attribute*

<pre>
root *
├── folder_1 *
│   ├── stream_1_1: float32_3
│   └── stream_1_2: uint8_3 *
├── folder_2 *
│   └── stream_2_1: int16_2
│   └── transients (event stream) *
│   └── loads (event stream)
├── folder_3 *
│   ├── folder_3_1 * 
│   │   └── stream_3_1_1: int32_3 *
│   └── stream_3_1: uint16_3
└── folder_4
    └── folder_4_1
</pre>


In [5]:
stream_3_1_1.elements[0].units="updated_units"
stream_1_2.description = "updated_description"
transient_events.event_fields = {'updated':'string'}
await node.data_stream_update(stream_3_1_1)
await node.data_stream_update(stream_1_2)
await node.event_stream_update(transient_events)
await save_schema("1_updated_schema.json")

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


*Move stream between two folders and move folder between two folders*

<pre>
root
├── folder_1
│   ├── stream_1_1: float32_3
│   └── <== removed ==>
├── folder_2
│   └── stream_2_1: int16_2
│   └── transients (event stream)
│   └── loads (event stream)
├── <== removed ==>
└── folder_4
    └── folder_4_1
    │   └── stream_1_2: uint8_3 <== moved to
    └── folder_3                  <== moved to
        ├── folder_3_1
        │   └── stream_3_1_1: int32_3
        └── stream_3_1: uint16_3
</pre>


In [6]:
await node.data_stream_move(stream_1_2, "/folder_4/folder_4_1")
await node.folder_move("/folder_3","/folder_4")
await save_schema("2_moved_schema.json")

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


*Delete folders and streams*

<pre>
root
├── folder_1
│   └── stream_1_1: float32_3
├── <== removed ==>
└── folder_4
    └── folder_4_1
    │   └── updated_name: uint8_3
    │   └── loads (event stream) <== moved to
    └── folder_3               
        ├── folder_3_1
        │   └── stream_3_1_1: int32_3
        └── <== removed ==>
</pre>


In [7]:
await node.event_stream_move(load_events,"/folder_4/folder_4_1")
await node.folder_delete("/folder_2")
await node.data_stream_delete(stream_3_1)
await save_schema("3_deleted_schema.json")

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


*Add new folders and streams*

<pre>
root
├── new <== new folder with streams
│   └── new_data_stream:float32_1
│   └── new_event_stream
├── folder_1
│   └── stream_1_1: float32_3
└── folder_4
    └── folder_4_1
    │   └── stream_2_1: uint8_3
    │   └── loads (event stream)
    └── folder_3               
        └── folder_3_1
            └── stream_3_1_1: int32_3
</pre>

In [8]:
new_data_stream = joule.api.DataStream("new_data_stream",elements=[joule.api.Element(f"{i}") for i in range(1)])
new_event_stream = joule.api.EventStream("new_event_stream",event_fields={"test":"string"})

load_events = await node.event_stream_create(new_event_stream, "/new")
new_data_stream = await node.data_stream_create(new_data_stream, "/new")
await save_schema("4_added_schema.json")

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


*Replace folders and streams with new ones that have the same name*

<pre>
root
├── new
│   └── new_data_stream:float32_1
│   └── new_event_stream <== new with same name
├── folder_1
│   └── stream_1_1: float32_3 <== new with same name
│   └── folder_3  <== moved              
│   │   └── folder_3_1
│   │       └── stream_3_1_1: int32_3
│   └── folder_4 <== moved
│       └── folder_4_1
│           └── stream_2_1: uint8_3
│           └── loads (event stream)
└── folder_4 <== new with same name
    └── stream_4: uint16
    └── events_4 (event stream)
    └── new_event_stream <== old version but moved
    </pre>

In [9]:
# remove old versions
await node.folder_move("/folder_4/folder_3","/folder_1")
await node.folder_move("/folder_4", "/folder_1")
await node.data_stream_delete("/folder_1/stream_1_1")
await node.event_stream_move("/new/new_event_stream","/folder_4")

# create new versions
new_event_stream = joule.api.EventStream("new_event_stream",event_fields={"test":"string"})
await node.event_stream_create(new_event_stream, "/new")

stream_1_1 = joule.api.DataStream("stream_1_1",elements=[joule.api.Element(x) for x in "xyz"])
await node.data_stream_create(stream_1_1,"/folder_1")

stream_4_1_1 = joule.api.DataStream("stream_4",datatype="uint16",elements=[joule.api.Element(f"{i}") for i in range(3)])
await node.data_stream_create(stream_4_1_1,"/folder_4")

events_4 = joule.api.EventStream("events_4",event_fields={"test":"string"})
await node.event_stream_create(events_4, "/folder_4")
await save_schema("5_modified_schema.json")
await node.close()

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
